In [2]:
import os
import platform
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, recall_score, precision_score, f1_score
from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import DeepFool

In [3]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Function to load the dataset based on the operating system
def load_dataset():
    os_type = platform.system()
    if os_type.startswith("Darwin"):
        dataset_path = "dataset/processed_data/main/final_cleaned_dataset.xlsx"
    else:
        dataset_path = "dataset\\processed_data\\main\\final_cleaned_dataset.xlsx"
    return pd.read_excel(dataset_path)

In [5]:
# Function to preprocess the dataset
def preprocess_dataset(data):
    # Remove unnecessary columns
    feature_value_temp = data.drop("Laboratory confirmed, since the beginning of the pandemic Hospitalized", axis=1)
    features = feature_value_temp.drop(["District", "Unnamed: 0", "Laboratory confirmed, since the beginning of the pandemic TOTAL",
                                       "Laboratory confirmed, since the beginning of the pandemic RecoveredA",
                                       "Laboratory confirmed, since the beginning of the pandemic Deceased"], axis=1)

    # Create target column
    data['flag'] = 3  # Initialize the 'flag' column with 0
    data.loc[data['Laboratory confirmed, since the beginning of the pandemic Hospitalized'] <= 449, 'flag'] = 2
    data.loc[data['Laboratory confirmed, since the beginning of the pandemic Hospitalized'] <= 290, 'flag'] = 1

    target_col = data["flag"]

    return features, target_col

In [6]:

# Function to train a Random Forest classifier
def train_random_forest_classifier(X_train, y_train):
    model = RandomForestClassifier(n_estimators=50, random_state=22)
    model.fit(X=X_train, y=y_train)
    return model


In [7]:

# Function to evaluate a classifier and print performance metrics
def evaluate_classifier(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mse)
    print(f"Root Mean Squared Error: {rmse}")
    print(f"Mean Squared Error: {mse}")
    print("R2 Score:", r2_score(y_test, y_pred))

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"F1 Score: {f1:.2f}")


In [8]:

# Main function
def main():
    # Load the dataset
    data = load_dataset()

    # Preprocess the dataset
    features, target_col = preprocess_dataset(data)

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(features, target_col, test_size=0.3, random_state=42)

    # Train a Random Forest classifier
    model = train_random_forest_classifier(X_train, y_train)

    # Create ART classifier for scikit-learn RandomForestClassifier
    art_classifier = SklearnClassifier(model=model)

    # Create ART DeepFool attack
    deepfool = DeepFool(classifier=art_classifier, max_iter=50, epsilon=1e-2, batch_size=1)

    # Generate adversarial samples with ART DeepFool attack
    x_train_adv = deepfool.generate(X_train.values)
    x_test_adv = deepfool.generate(X_test.values)

    # Save the adversarial test data
    np.save('x_test_adv.npy', x_test_adv)

    # Evaluate and print the model scores
    print("Normal Training Score:")
    evaluate_classifier(model, X_train, y_train)

    print("\nAdversarial Training Score:")
    evaluate_classifier(model, x_train_adv, y_train)

    print("\nNormal Test Score:")
    evaluate_classifier(model, X_test, y_test)

    print("\nAdversarial Test Score:")
    evaluate_classifier(model, x_test_adv, y_test)


In [11]:
main()

EstimatorError: DeepFool requires an estimator derived from <class 'art.estimators.estimator.BaseEstimator'> and <class 'art.estimators.classification.classifier.ClassGradientsMixin'>, the provided classifier is an instance of <class 'art.estimators.classification.scikitlearn.ScikitlearnRandomForestClassifier'> and is derived from (<class 'art.estimators.classification.scikitlearn.ScikitlearnClassifier'>,).